# Water System Optimization Experiments Using Genetic Algorithms

This notebook demonstrates the optimization of a water system using genetic algorithms. We'll optimize reservoir release parameters and hydroworks distribution parameters to minimize supply deficits while considering operational constraints.

We will change different parameters of the GA (Genetic Algorithm) like population size, number of generations, crossover probability and mutation probability in order to find the best suited GA parameter set for the Zarafshan River Basin Model.

## Setup

First, let's import the required libraries and set up our parameters.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from water_system import  WaterSystem, SupplyNode, StorageNode, DemandNode, SinkNode, HydroWorks, Edge, WaterSystemVisualizer, MultiGeneticOptimizer
import os
import sys

# Add the project directory to the system path
project_dir = 'C:/Users/giann/water_system_simulation_optimization'
if project_dir not in sys.path:
    sys.path.append(project_dir)

# Import the run sample tests function
from ZRB_system import run_ipynb_optimization
# Import the create seasonal ZRB system function
from ZRB_system import create_seasonal_ZRB_system
from ZRB_system import save_optimized_parameters

## ZRB system

The goal is to optimize annual reservoir and hydrowork parameter for the ZRB system. The following code block shows how the GA is called.

In [2]:
start_year = 2017
start_month = 1
num_time_steps = 12*3
ngen = 5
pop_size = 5
cxpb = 0.5
mutpb = 0.2

ZRB_system= create_seasonal_ZRB_system(start_year, start_month, num_time_steps)

optimizer = MultiGeneticOptimizer(
        base_system=ZRB_system,
        start_year=start_year,
        start_month=start_month,
        num_time_steps=num_time_steps,
        ngen=ngen,
        population_size=pop_size,
        cxpb=cxpb,
        mutpb=mutpb
    )

results = optimizer.optimize()

print("\nOptimization Results:")
print("-" * 50)
print(f"Message: {results['message']}")
print(f"Population size: {results['population_size']}")
print(f"Generations: {results['generations']}")
print(f"Corss-over probability: {results['crossover_probability']}")
print(f"Mutation probability: {results['mutation_probability']}")
print(f"Final objective value: {results['objective_value']:,.0f} m³")

optimizer.plot_convergence()

gen	nevals	min        	avg        	std        
0  	5     	1.05574e+12	1.93327e+12	5.31528e+11
1  	2     	1.05574e+12	1.16305e+12	2.14614e+11
2  	4     	1.05574e+12	1.05574e+12	0          
3  	1     	1.05574e+12	1.29041e+12	4.69334e+11
4  	4     	8.13944e+11	1.00738e+12	9.67187e+10
5  	2     	8.13944e+11	1.19955e+12	5.72634e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 5
Generations: 5
Corss-over probability: 0.5
Mutation probability: 0.2
Final objective value: 813,944,344,660 m³


## Optuna

Optuna is a modern hyperparameter optimization framework that automates the process of finding the best parameters for a machine learning model, algorithm, or any other optimization task. It is highly flexible, supports advanced techniques like pruning, and is suitable for a variety of optimization scenarios, including tuning parameters for genetic algorithms (GA).

1. Efficient Sampling:
- Uses techniques like Tree-structured Parzen Estimator (TPE) and other samplers to efficiently explore the search space.
- Focuses on promising areas of the search space instead of blindly testing all combinations.

2. Pruning:
- Dynamically stops unpromising trials (parameter combinations) during execution, saving computation time.

3. Dynamic Search Space:
- Allows the search space to be updated based on intermediate results.

4. Distributed Optimization:
- Scales to multiple machines or processes to perform trials in parallel.

5. Visualization:
- Provides tools to analyze the optimization process, including parameter importance, convergence, and search space exploration.


### Advantages of Optuna
- Automated Tuning: Efficiently finds good parameters without manual intervention.
- Flexibility: Supports dynamic and conditional search spaces.
- Scalability: Easily integrates with parallel and distributed systems.
- Ease of Use: Intuitive API and great visualization tools.


### Let's try...
Keep it small and simple just to see if it works.

In [4]:
import optuna

def objective(trial):
    population = trial.suggest_int('pop_size', 20, 30)
    generations = trial.suggest_int('generations', 5, 6)
    cxpb = trial.suggest_float('cxpb', 0.5, 0.9)
    mutpb = trial.suggest_float('mutpb', 0.1, 0.5)

    optimization_results = run_ipynb_optimization(start_year, start_month, num_time_steps, generations, population, cxpb, mutpb)
    return optimization_results['objective_value']

# Create a study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Print the best parameters and fitness value
print("Best Parameters:", study.best_params)
print("Best Objective Value:", study.best_value)


[I 2025-01-21 14:48:59,257] A new study created in memory with name: no-name-2091777a-1226-44af-99ad-1d6c9245f2ef
c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	min        	avg        	std        
0  	25    	6.41746e+11	1.57927e+12	5.53253e+11
1  	23    	5.59648e+11	9.7894e+11 	3.16285e+11
2  	19    	4.49648e+11	7.78308e+11	3.12584e+11
3  	16    	3.83382e+11	7.37531e+11	3.83131e+11
4  	17    	3.83382e+11	6.49033e+11	3.19842e+11
5  	18    	3.83382e+11	4.90157e+11	1.96528e+11


[I 2025-01-21 14:49:04,909] Trial 0 finished with value: 382701233390.0831 and parameters: {'pop_size': 25, 'generations': 6, 'cxpb': 0.7776771417084161, 'mutpb': 0.17629968664587295}. Best is trial 0 with value: 382701233390.0831.


6  	18    	3.82701e+11	5.37879e+11	3.84523e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 25
Generations: 6
Corss-over probability: 0.7776771417084161
Mutation probability: 0.17629968664587295
Final objective value: 382,701,233,390 m³
Optimization results saved to ./GA_experiments/optimized_parameters_ZRB_ngen6_pop25_cxpb0.7776771417084161_mutpb0.17629968664587295.json


c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


...

6  	25    	3.29581e+11	5.03668e+11	4.38593e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 30
Generations: 6
Corss-over probability: 0.7596248128903365
Mutation probability: 0.27414992239510605
Final objective value: 329,581,458,276 m³
Optimization results saved to ./GA_experiments/optimized_parameters_ZRB_ngen6_pop30_cxpb0.7596248128903365_mutpb0.27414992239510605.json


c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	min       	avg        	std        
0  	23    	8.1923e+11	1.47825e+12	4.82652e+11
1  	16    	6.22657e+11	9.96656e+11	2.6693e+11 
2  	10    	5.9132e+11 	7.93733e+11	9.0341e+10 
3  	16    	5.2714e+11 	7.80105e+11	2.61184e+11
4  	16    	5.2714e+11 	7.2949e+11 	3.32341e+11
5  	20    	3.99843e+11	7.2244e+11 	3.58678e+11


[I 2025-01-21 14:49:29,452] Trial 4 finished with value: 399843285299.2107 and parameters: {'pop_size': 23, 'generations': 6, 'cxpb': 0.5767117847438494, 'mutpb': 0.16999438049420645}. Best is trial 3 with value: 329581458276.27325.


6  	15    	3.99843e+11	6.91596e+11	2.58187e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 23
Generations: 6
Corss-over probability: 0.5767117847438494
Mutation probability: 0.16999438049420645
Final objective value: 399,843,285,299 m³
Optimization results saved to ./GA_experiments/optimized_parameters_ZRB_ngen6_pop23_cxpb0.5767117847438494_mutpb0.16999438049420645.json
Best Parameters: {'pop_size': 30, 'generations': 6, 'cxpb': 0.7596248128903365, 'mutpb': 0.27414992239510605}
Best Objective Value: 329581458276.27325


#### Visualize Results

In [5]:
import plotly
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_contour, plot_intermediate_values, plot_timeline, plot_slice, plot_edf
# Plot optimization history
plot_optimization_history(study).show()

# Plot parameter importances
plot_param_importances(study).show()

# different plots
plot_edf(study).show()
plot_slice(study).show()
plot_contour(study).show()
plot_intermediate_values(study).show()
plot_timeline(study).show()


[W 2025-01-17 11:43:57,634] Param generations unique value length is less than 2.
[W 2025-01-17 11:43:57,635] Param generations unique value length is less than 2.
[W 2025-01-17 11:43:57,635] Param generations unique value length is less than 2.
[W 2025-01-17 11:43:57,635] Param generations unique value length is less than 2.
[W 2025-01-17 11:43:57,638] Param generations unique value length is less than 2.
[W 2025-01-17 11:43:57,638] Param generations unique value length is less than 2.


[W 2025-01-17 11:43:57,817] You need to set up the pruning feature to utilize `plot_intermediate_values()`


### Let's go a little bit bigger...

In [9]:
def objective(trial):
    population = trial.suggest_int('pop_size', 50, 500)
    generations = trial.suggest_int('generations', 20, 100)
    cxpb = trial.suggest_float('cxpb', 0.5, 0.99)
    mutpb = trial.suggest_float('mutpb', 0.1, 0.8)

    optimization_results = run_ipynb_optimization(start_year, start_month, num_time_steps, generations, population, cxpb, mutpb)
    return optimization_results['objective_value']

# Create a study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Print the best parameters and fitness value
print("Best Parameters:", study.best_params)
print("Best Objective Value:", study.best_value)

[I 2025-01-17 11:47:32,625] A new study created in memory with name: no-name-92526945-950e-45f4-9c93-3be954f9bd0c
c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



gen	nevals	min        	avg        	std        
0  	137   	4.98353e+11	1.54346e+12	5.54831e+11
1  	122   	3.73466e+11	1.03802e+12	4.24886e+11
2  	132   	4.03304e+11	8.68394e+11	4.89584e+11
3  	135   	3.84977e+11	7.93404e+11	4.80321e+11
4  	131   	3.77205e+11	6.35043e+11	3.14305e+11
5  	135   	3.47728e+11	7.05383e+11	4.68817e+11
6  	133   	3.47728e+11	7.07078e+11	5.35138e+11
7  	132   	3.35248e+11	6.77234e+11	4.87035e+11
8  	135   	3.47728e+11	6.93854e+11	5.56332e+11
9  	134   	3.2289e+11 	6.86316e+11	5.93781e+11
10 	136   	3.22116e+11	6.01538e+11	4.74128e+11
11 	135   	3.18538e+11	5.66542e+11	4.28731e+11
12 	134   	3.17017e+11	6.83374e+11	5.81444e+11
13 	133   	3.17017e+11	6.02072e+11	4.85767e+11
14 	135   	3.17017e+11	5.1329e+11 	3.87552e+11
15 	133   	3.17017e+11	5.5649e+11 	4.21265e+11
16 	134   	3.17017e+11	6.38976e+11	5.55351e+11
17 	132   	3.17017e+11	5.87581e+11	4.95807e+11
18 	136   	3.17017e+11	5.20785e+11	4.30068e+11
19 	136   	3.16827e+11	6.68891e+11	5.52487e+11
20 	131   	3.

[I 2025-01-17 12:04:22,797] Trial 0 finished with value: 240128113719.85934 and parameters: {'pop_size': 137, 'generations': 51, 'cxpb': 0.9599629930779292, 'mutpb': 0.4702518952396939}. Best is trial 0 with value: 240128113719.85934.


51 	133   	2.40128e+11	5.93696e+11	5.44637e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 137
Generations: 51
Corss-over probability: 0.9599629930779292
Mutation probability: 0.4702518952396939
Final objective value: 240,128,113,720 m³
Optimization results saved to ./GA_experiments/optimized_parameters_ZRB_ngen51_pop137_cxpb0.9599629930779292_mutpb0.4702518952396939.json


c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



...

49 	385   	1.67757e+11	4.02397e+11	4.89189e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 475
Generations: 49
Corss-over probability: 0.7120547226671534
Mutation probability: 0.2517107107329721
Final objective value: 167,756,904,009 m³
Optimization results saved to ./GA_experiments/optimized_parameters_ZRB_ngen49_pop475_cxpb0.7120547226671534_mutpb0.2517107107329721.json


c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\giann\Anaconda\envs\envMT\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



gen	nevals	min        	avg        	std        
0  	114   	4.08857e+11	1.42956e+12	5.29717e+11
1  	95    	4.08857e+11	9.62353e+11	3.33791e+11
2  	99    	3.90612e+11	8.88687e+11	4.91131e+11
3  	99    	3.90612e+11	7.53152e+11	4.28334e+11
4  	92    	3.65701e+11	6.98339e+11	4.27783e+11
5  	92    	3.65701e+11	6.63747e+11	4.63598e+11
6  	98    	3.76674e+11	6.65802e+11	4.33165e+11
7  	94    	3.62588e+11	6.22212e+11	4.28145e+11
8  	91    	3.46409e+11	5.86884e+11	3.66162e+11
9  	97    	3.55971e+11	6.32332e+11	4.93111e+11
10 	89    	3.55971e+11	6.71628e+11	4.55255e+11
11 	98    	3.05227e+11	6.93307e+11	4.55776e+11
12 	94    	3.05227e+11	5.78792e+11	4.03445e+11
13 	96    	2.88569e+11	5.81295e+11	4.38289e+11
14 	94    	2.87138e+11	5.70704e+11	4.3455e+11 
15 	89    	2.78734e+11	5.99271e+11	5.49868e+11
16 	89    	2.60155e+11	5.5144e+11 	4.6274e+11 
17 	101   	2.60155e+11	5.46375e+11	4.62537e+11
18 	100   	2.46133e+11	5.89271e+11	4.93976e+11
19 	103   	2.46133e+11	5.72668e+11	4.60925e+11
20 	95    	2.

[I 2025-01-17 15:19:33,593] Trial 4 finished with value: 222397010336.02673 and parameters: {'pop_size': 114, 'generations': 30, 'cxpb': 0.7084377768303122, 'mutpb': 0.4610073063185568}. Best is trial 2 with value: 158523407688.37564.


30 	97    	2.22397e+11	5.71992e+11	5.17755e+11

Optimization Results:
--------------------------------------------------
Message: Optimization completed successfully
Population size: 114
Generations: 30
Corss-over probability: 0.7084377768303122
Mutation probability: 0.4610073063185568
Final objective value: 222,397,010,336 m³
Optimization results saved to ./GA_experiments/optimized_parameters_ZRB_ngen30_pop114_cxpb0.7084377768303122_mutpb0.4610073063185568.json
Best Parameters: {'pop_size': 451, 'generations': 56, 'cxpb': 0.8204034986005839, 'mutpb': 0.16513645373298075}
Best Objective Value: 158523407688.37564


In [10]:
import plotly
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_contour, plot_intermediate_values, plot_timeline, plot_slice, plot_edf
# Plot optimization history
plot_optimization_history(study).show()

# Plot parameter importances
plot_param_importances(study).show()

# different plots
plot_edf(study).show()
plot_slice(study).show()
plot_contour(study).show()
plot_intermediate_values(study).show()
plot_timeline(study).show()

[W 2025-01-17 15:44:25,746] You need to set up the pruning feature to utilize `plot_intermediate_values()`
